<table width = "100%">
  <tr style="background-color:white;">
    <!-- QWorld Logo -->
    <td style="text-align:left;width:200px;"> 
        <a href="https://qworld.net/" target="_blank"><img src="../images/QWorld.png"> </a></td>
    <td style="text-align:right;vertical-align:bottom;font-size:16px;"> 
        Prepared by <a href="https://www.cmpe.boun.edu.tr/~ozlem.salehi/" target="_blank"> Özlem Salehi </a> </td>
    </tr> 
 </table>
 
<hr>

# <font color="blue"> Solutions for </font> D-Wave Hybrid Solvers

<a id="Task1"></a>


### Task 1

Solve the TSP problem for the following graph with 100 nodes using the hybrid BQM solver. Interpret the results.

In [2]:
import numpy as np
import networkx as nx

np.random.seed(45)
N = 500
G1 = nx.complete_graph(N)
for u, v in G1.edges():
    G1[u][v]["weight"] = np.random.randint(1, 5)

### Solution

Let us copy the codes for creating BQM for TSP and checking the results.

In [3]:
from dimod import BQM

def tsp_bqm(G, P):
    
    N = len(G.nodes)
    bqm = BQM("BINARY")
    for i in range(N):
        for j in range(N):
            if i!=j:
                for t in range(N-1):
                    bqm.add_quadratic(f"x_{i}_{t}", f"x_{j}_{t+1}", G[i][j]["weight"])

                #Remember that we were assuming N=0 in the sum
                bqm.add_quadratic(f"x_{i}_{N-1}", f"x_{j}_{0}", G[i][j]["weight"])
    #Add the first constraint
    for t in range(N):
        c1 = [(f"x_{i}_{t}", 1) for i in range(N)] #coefficient list
        bqm.add_linear_equality_constraint(c1, constant=-1, lagrange_multiplier=P)
    #Add the second constraint
    for i in range(N):
        c2 = [(f"x_{i}_{t}", 1) for t in range(N)]
        bqm.add_linear_equality_constraint(c2, constant=-1, lagrange_multiplier=P)
    return bqm

def is_sample_feasible(sample,N):
    for i in range(N):
        if sum(sample[f"x_{i}_{t}"] for t in range(N))!=1:
            return False
    for t in range(N):
        if sum(sample[f"x_{i}_{t}"] for i in range(N))!=1:
            return False
    return True

def sample_to_path(sample,N):
    path = []
    for t in range(N):
        for i in range(N):
            if sample[f"x_{i}_{t}"] == 1:
                path.append(i)
    return path

We create the BQM and an instance of the solver.

In [ ]:
from dwave.system import LeapHybridBQMSampler

bqm = tsp_bqm(G1, 5)
sampler = LeapHybridBQMSampler()                         

We run the problem on the solver. 

In [ ]:
sampleset = sampler.sample(bqm)

We can see that it took 40 seconds to run the problem.

In [ ]:
sampleset.info

Let us now check the feasibility of the sample and convert it into a route.

In [ ]:
print(is_sample_feasible(sampleset.first.sample,N))
print(sample_to_path(sampleset.first.sample,N),sampleset.first.energy)

<a id="Task2"></a>


### Task 2

Write a funcion named `tsp_cqm` that takes as parameter a networkx graph $G$ and returns the constrained quadratic model for TSP problem. 


### Solution

In [1]:
from dimod import BQM, CQM, Binary

def tsp_cqm(G):
    N = len(G.nodes)
    bqm = BQM("BINARY")
    for i in range(N):
        for j in range(N):
            if i!=j:
                for t in range(N-1):
                    bqm.add_quadratic(f"x_{i}_{t}", f"x_{j}_{t+1}", G[i][j]["weight"])

                #Remember that we were assuming N=0 in the sum
                bqm.add_quadratic(f"x_{i}_{N-1}", f"x_{j}_{0}", G[i][j]["weight"])
    cqm = CQM()
    cqm.set_objective(bqm)
    
    #Add the first constraint
    for t in range(N):
        c1 = sum(Binary(f"x_{i}_{t}") for i in range(N)) 
        cqm.add_constraint(c1 == 1, label = f"C1_{t}")
    #Add the second constraint
    for i in range(N):
        c2 = sum(Binary(f"x_{i}_{t}") for t in range(N)) 
        cqm.add_constraint(c2 == 1, label = f"C2_{i}")
    return cqm

<a id="Task3"></a>


### Task 3

Using the function you have written in Task 2, obtain the cqm for the given graph and solve it using the hybrid CQM solver.

In [2]:
import numpy as np
import networkx as nx

np.random.seed(45)
N = 100
G1 = nx.complete_graph(N)
for u, v in G1.edges():
    G1[u][v]["weight"] = np.random.randint(1, 5)

### Solution

First we obtain the cqm.

In [3]:
cqm = tsp_cqm(G1)

We create an instance of the sampler and run our problem.

In [4]:
from dwave.system import LeapHybridCQMSampler

In [5]:
cqm_solver = LeapHybridCQMSampler()                         
sampleset = cqm_solver.sample_cqm(cqm)

Let us check sampleset info. It took around 8 seconds to solve the problem.

In [6]:
print(sampleset.info)

{'constraint_labels': ['C1_38', 'C2_69', 'C1_94', 'C1_9', 'C2_22', 'C2_10', 'C1_14', 'C1_67', 'C1_31', 'C1_78', 'C2_64', 'C1_96', 'C1_19', 'C1_47', 'C2_81', 'C2_4', 'C2_24', 'C1_18', 'C2_16', 'C1_33', 'C2_75', 'C2_80', 'C2_19', 'C1_45', 'C1_68', 'C2_57', 'C1_49', 'C2_2', 'C1_64', 'C2_85', 'C2_37', 'C2_53', 'C1_5', 'C1_87', 'C2_14', 'C2_93', 'C1_40', 'C2_96', 'C2_0', 'C2_54', 'C2_44', 'C2_36', 'C1_95', 'C2_34', 'C1_57', 'C2_27', 'C1_72', 'C2_98', 'C1_77', 'C2_50', 'C1_0', 'C2_13', 'C1_20', 'C2_33', 'C1_43', 'C1_71', 'C2_74', 'C1_15', 'C2_92', 'C1_22', 'C1_79', 'C2_83', 'C1_52', 'C2_26', 'C2_17', 'C1_21', 'C1_73', 'C1_89', 'C1_90', 'C1_42', 'C1_63', 'C2_70', 'C2_65', 'C1_99', 'C2_6', 'C2_38', 'C1_60', 'C2_9', 'C1_1', 'C1_25', 'C2_18', 'C2_21', 'C2_42', 'C2_97', 'C1_3', 'C2_55', 'C1_70', 'C2_5', 'C1_93', 'C2_48', 'C1_23', 'C2_61', 'C1_13', 'C1_41', 'C2_35', 'C1_65', 'C1_74', 'C2_40', 'C2_88', 'C2_3', 'C1_8', 'C1_98', 'C2_94', 'C1_92', 'C2_46', 'C1_62', 'C2_25', 'C1_24', 'C1_81', 'C2_91', 

The first returned sample is not feasible as can be seen below.

In [18]:
print(sampleset.first.is_feasible)

False


We can filter the feasible samples and pick the lowest energy one.

In [17]:
best_sample = sampleset.filter(lambda d: d.is_feasible).first
print(sample_to_path(best_sample.sample,N),best_sample.energy)

[42, 69, 21, 29, 53, 40, 66, 14, 83, 72, 67, 6, 31, 65, 98, 89, 9, 25, 92, 12, 90, 96, 61, 34, 24, 74, 45, 13, 8, 76, 41, 97, 10, 88, 80, 95, 84, 85, 56, 48, 81, 15, 71, 54, 49, 38, 20, 60, 70, 64, 3, 73, 86, 35, 5, 55, 2, 68, 22, 75, 93, 17, 47, 36, 99, 26, 44, 18, 77, 23, 50, 4, 1, 52, 43, 82, 11, 16, 33, 51, 91, 57, 94, 62, 39, 46, 27, 37, 59, 58, 7, 28, 79, 32, 19, 0, 78, 30, 87, 63] 104.0


In [19]:
sampleset_20 = cqm_solver.sample_cqm(cqm, time_limit = 20)

In [20]:
sampleset_20.info

{'constraint_labels': ['C1_38',
  'C2_69',
  'C1_94',
  'C1_9',
  'C2_22',
  'C2_10',
  'C1_14',
  'C1_67',
  'C1_31',
  'C1_78',
  'C2_64',
  'C1_96',
  'C1_19',
  'C1_47',
  'C2_81',
  'C2_4',
  'C2_24',
  'C1_18',
  'C2_16',
  'C1_33',
  'C2_75',
  'C2_80',
  'C2_19',
  'C1_45',
  'C1_68',
  'C2_57',
  'C1_49',
  'C2_2',
  'C1_64',
  'C2_85',
  'C2_37',
  'C2_53',
  'C1_5',
  'C1_87',
  'C2_14',
  'C2_93',
  'C1_40',
  'C2_96',
  'C2_0',
  'C2_54',
  'C2_44',
  'C2_36',
  'C1_95',
  'C2_34',
  'C1_57',
  'C2_27',
  'C1_72',
  'C2_98',
  'C1_77',
  'C2_50',
  'C1_0',
  'C2_13',
  'C1_20',
  'C2_33',
  'C1_43',
  'C1_71',
  'C2_74',
  'C1_15',
  'C2_92',
  'C1_22',
  'C1_79',
  'C2_83',
  'C1_52',
  'C2_26',
  'C2_17',
  'C1_21',
  'C1_73',
  'C1_89',
  'C1_90',
  'C1_42',
  'C1_63',
  'C2_70',
  'C2_65',
  'C1_99',
  'C2_6',
  'C2_38',
  'C1_60',
  'C2_9',
  'C1_1',
  'C1_25',
  'C2_18',
  'C2_21',
  'C2_42',
  'C2_97',
  'C1_3',
  'C2_55',
  'C1_70',
  'C2_5',
  'C1_93',
  'C2_48',


In [21]:
best_sample_20 = sampleset.filter(lambda d: d.is_feasible).first
print(sample_to_path(best_sample.sample,N),best_sample.energy)

[42, 69, 21, 29, 53, 40, 66, 14, 83, 72, 67, 6, 31, 65, 98, 89, 9, 25, 92, 12, 90, 96, 61, 34, 24, 74, 45, 13, 8, 76, 41, 97, 10, 88, 80, 95, 84, 85, 56, 48, 81, 15, 71, 54, 49, 38, 20, 60, 70, 64, 3, 73, 86, 35, 5, 55, 2, 68, 22, 75, 93, 17, 47, 36, 99, 26, 44, 18, 77, 23, 50, 4, 1, 52, 43, 82, 11, 16, 33, 51, 91, 57, 94, 62, 39, 46, 27, 37, 59, 58, 7, 28, 79, 32, 19, 0, 78, 30, 87, 63] 104.0
